<a href="https://colab.research.google.com/github/monu322/CE888/blob/main/Assg2/EmotionDetection/Topic_number_for_Emotion_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyLDAvis

In [3]:
import pickle
import gensim
import pyLDAvis
import pyLDAvis.gensim_models
import spacy
import pandas as pd
import nltk; nltk.download('stopwords')
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import re
import warnings
from pprint import pprint

warnings.filterwarnings("ignore")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
#importing emotion dataset

tweets_emotion=pd.read_fwf('https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/emotion/train_text.txt', header=None)
print('Training tweets', tweets_emotion.shape)

tweets_emotion_labels=pd.read_fwf('https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/emotion/train_labels.txt', header=None)
print('Training labels', tweets_emotion_labels.shape)

tweets_emotion_test=pd.read_fwf('https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/emotion/test_text.txt', header=None)
print('Test tweets',tweets_emotion_test.shape)

tweets_emotion_test_labels=pd.read_fwf('https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/emotion/test_labels.txt', header=None)
print('Test labels',tweets_emotion_test_labels.shape)


Training tweets (3257, 1)
Training labels (3257, 1)
Test tweets (1421, 1)
Test labels (1421, 1)


In [6]:
tweets_emotion.head()

,0
0,“Worry is a down payment on a problem you may ...
1,My roommate: it's okay that we can't spell bec...
2,No but that's so cute. Atsu was probably shy a...
3,Rooneys fucking untouchable isn't he? Been fuc...
4,it's pretty depressing when u hit pan on ur fa...


In [8]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

def strip_newline(series):
    return [review.replace('\n','') for review in series]

tweets_emotion[0] = strip_newline(tweets_emotion[0])
tweets_emotion[0][20:42].values

array(['Love is when all your happiness and all your sadness and all your feelings are dependent on another person.',
       'It’s possible changing meds is best not done while under stress. Difficult to tell what part of despair is circumstantial, what is drugs.',
       'im also definitely still bitter about the yellow ranger not being asian, but asian representation in hollywood is essentially a shrug anyway',
       '@user The irony is that those protesting about this kind of stuff are the Orwellian nightmare they think they’re fighting against.',
       "@user @user nah way that's horrible",
       '@user I think just becz u have so much terror in pak nd urself being  a leader u forgot d difference btw a leader nd terrorist !',
       'angel delight is my everything',
       'Puzzle investing opening portland feodal population is correlative straight a snorting infuriate: XLzjYhG',
       "I believe I'm gonna start singing in my snap stories on the tractor. Switch it up a little b

In [9]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

tweet_words = list(sent_to_words(tweets_emotion[0]))

tweet_words[21][:10]

['it',
 'possible',
 'changing',
 'meds',
 'is',
 'best',
 'not',
 'done',
 'while',
 'under']

In [10]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

tweet_words = remove_stopwords(tweet_words)

def bigrams(words, bi_min=15, tri_min=10):
    bigram = gensim.models.Phrases(words, min_count = bi_min)
    trigram = gensim.models.Phrases(bigram[words], min_count = tri_min)
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    trigram_mod = gensim.models.phrases.Phraser(trigram)
    return bigram_mod, trigram_mod

bigram_tr, trigram_tr = bigrams(tweet_words)

print(trigram_tr[bigram_tr[tweet_words[0]]][:200])

['worry', 'payment', 'problem', 'may', 'never', 'joyce', 'meyer', 'motivation', 'leadership', 'worry']


In [11]:
nlp = spacy.load('en', disable=['parser', 'ner'])

warnings.filterwarnings("ignore")

def lemmatization(texts, allowed_postags=['NOUN','ADJ','VERB','ADV']):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

trigrams_tr = [trigram_tr[bigram_tr[review]] for review in tweet_words]

lemma_lg = lemmatization(trigrams_tr)

In [12]:
lemma_lg[0][:20]

id2word_lg = gensim.corpora.Dictionary(lemma_lg)
id2word_lg.filter_extremes(no_below=10, no_above=0.35)
id2word_lg.compactify()
id2word_lg.save('train_dict_lg')
corpus_lg = [id2word_lg.doc2bow(text) for text in lemma_lg]

corpus_lg[22][:10]

[(59, 1), (60, 1), (61, 1), (62, 1)]

In [13]:
from gensim.models import HdpModel
hdp = HdpModel(corpus_lg, id2word_lg, chunksize=10000)

len(hdp.print_topics())

20

In [14]:
hdp.print_topics(num_topics=20)

[(0,
  '0.013*give + 0.013*blue + 0.013*furious + 0.012*pout + 0.012*hate + 0.012*anger + 0.011*leave + 0.011*hair + 0.011*head + 0.011*turn'),
 (1,
  '0.015*instead + 0.014*sleep + 0.014*free + 0.013*rabid + 0.013*suppose + 0.012*miss + 0.012*story + 0.011*mad + 0.011*better + 0.011*joke'),
 (2,
  '0.014*good + 0.014*high + 0.013*unhappy + 0.012*right + 0.011*sick + 0.010*literally + 0.009*white + 0.009*time + 0.009*movie + 0.009*joke'),
 (3,
  '0.013*eat + 0.012*always + 0.011*fear + 0.011*week + 0.010*funny + 0.010*boy + 0.010*people + 0.009*fucking + 0.009*need + 0.009*alarm'),
 (4,
  '0.020*bitch + 0.013*man + 0.012*hilarity + 0.011*sit + 0.011*boy + 0.011*level + 0.010*act + 0.010*seriously + 0.010*actually + 0.010*true'),
 (5,
  '0.020*terrorism + 0.014*country + 0.014*hope + 0.013*turn + 0.012*discourage + 0.011*horror + 0.011*wait + 0.010*move + 0.010*start + 0.010*lol'),
 (6,
  '0.016*lose + 0.015*sadly + 0.013*sober + 0.013*keep + 0.011*house + 0.011*black + 0.010*hilarity +